<a href="https://colab.research.google.com/github/zeoses/movielenCF/blob/master/CFRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
from sklearn.cluster import KMeans
import math
from tqdm import tqdm 

In [0]:
# Read data with header None
data = pd.read_csv('/content/gdrive//My Drive/Colab Notebooks/u.data',delimiter='\t',header=None)
ClusterCount=3
RatioTrain2Test=0.8

In [0]:
# Create User_item Matrix
userSize=max(data[0][:])
itemSize=max(data[1][:])

userItemMatrix=np.zeros((userSize,itemSize))

re=(data[1][:].size)
for i in range(0,re):
    userItemMatrix[(data[0][i])-1][(data[1][i])-1]=int(data[2][i]);
    
#np.savetxt("D:\\foo.csv", userItemMatrix, delimiter="\t")


In [0]:
# Split Data to Train and Test
trainSize =int(np.floor(userSize * RatioTrain2Test));
testSize = int(userSize-trainSize);

trainValue=[];
testValue=[];


trainValue = userItemMatrix[testSize:,:]
testValue = userItemMatrix[:testSize,:];



trainValue=np.array(trainValue)
testValue=np.array(testValue)  

In [0]:
# Clustring Data For Kmeans Alghoritm

kmeans = KMeans(n_clusters=ClusterCount)
kmeans.fit(trainValue)
resultOfKmeans=kmeans.labels_
centerOfKmeansResult=kmeans.cluster_centers_

# Predict For TestData
prdct = kmeans.predict(testValue)


In [0]:
 
#avarege Parameters 
ATP=0.0;
ATN=0.0;
AFP=0.0;
AFN=0.0;

#person Parameters
PTP=0.0;
PTN=0.0;
PFP=0.0;
PFN=0.0;


# compute similarity
for i in tqdm(range(testSize)):
  
  
    ClusterNumber=prdct[i];
    ClusterMember=trainValue[resultOfKmeans==ClusterNumber,]
    similarity1=np.zeros(ClusterMember.shape[0])
    aveForTestItem=np.average(testValue[i,testValue[i]>0]);
    
#     Find  Pearson similarity
    for j in range(ClusterMember.shape[0]):
        aveForMember=np.average(ClusterMember[j,ClusterMember[j]>0]);
        sumOfNumerator=0
        Denominator1=0
        Denominator=0
        Denominator2=0
        for u in range(trainValue.shape[1]):
           if ClusterMember[j,u]>0 and testValue[i,u]>0:
                sumOfNumerator=sumOfNumerator+((ClusterMember[j,u]-aveForMember)*(testValue[i,u]-aveForTestItem));
                Denominator1=Denominator1+((ClusterMember[j,u]-aveForMember)**2)
                Denominator2=Denominator2+((testValue[i,u]-aveForTestItem)**2)

        Denominator=(math.sqrt(Denominator1))*(math.sqrt(Denominator2))
        if(Denominator!=0):
            sim=sumOfNumerator/Denominator
        else:
            sim=0

        similarity1[j]=sim

    
    # Rate Predict 
    pridictAverage=np.zeros(trainValue.shape[1])     
    pridictPearson=np.zeros(trainValue.shape[1])

    for ll in range(ClusterMember.shape[1]):
        numeratorPrdicPearson=0
        denominatorPrdicPearson=0
        for l in range(ClusterMember.shape[0]):
            if ClusterMember[l,ll]>0 and similarity1[l]>0: 
              numeratorPrdicPearson=numeratorPrdicPearson+(similarity1[l]*(ClusterMember[l,ll]-np.average(ClusterMember[l,ClusterMember[l]>0])))
              denominatorPrdicPearson=denominatorPrdicPearson+similarity1[l];
        if denominatorPrdicPearson>0:
          pridictPearson[ll]=(aveForTestItem+(numeratorPrdicPearson/denominatorPrdicPearson))
        else:
          pridictPearson[ll]=0

        pridictAverage[ll]=(np.average(ClusterMember[ClusterMember[:,ll]>0,ll]))
        #pridictAverage[ll]=(np.average(trainValue[trainValue[:,ll]>0,ll]))
        if np.isnan(pridictAverage[ll]):
          pridictAverage[ll]=0

        TargetRate=testValue[i,ll]
        PredictRateAVE=pridictAverage[ll]
        PredictRatePear=pridictPearson[ll]

        if TargetRate !=0 and PredictRatePear !=0:
          if TargetRate == 5 and PredictRatePear >= 4.5:
            PTP = PTP + 1
          elif  TargetRate < 5 and PredictRatePear >= 4.5:
            PFP = PFP + 1
          elif  TargetRate == 5 and PredictRatePear < 4.5:
            PFN = PFN + 1
          elif  TargetRate < 5 and PredictRatePear < 4.5:
            PTN = PTN + 1


        if TargetRate !=0 and PredictRateAVE !=0:
          if TargetRate == 5 and PredictRateAVE >= 4.5:
            ATP = ATP + 1;
          elif  TargetRate < 5 and PredictRateAVE >= 4.5:
            AFP = AFP + 1;
          elif  TargetRate == 5 and PredictRateAVE < 4.5:
            AFN = AFN + 1;
          elif  TargetRate < 5 and PredictRateAVE < 4.5:
            ATN = ATN + 1

      
AT = ATP + AFP + ATN + AFN
PT = PTP + PFP + PTN + PFN
#Compute value for accuracy and precision and recall 

Accuracy = ((ATP + ATN) / AT)*100; 
Precision = (ATP / (ATP + AFP))*100;
Recall = (ATP/(ATP+AFN))*100; 

PAccuracy1 = ((PTP + PTN) / PT)*100;
PPrecision1 = (PTP / (PTP + PFP))*100;
PRecall1 = (PTP/(PTP+PFN))*100;
AverageResult=[Accuracy,Precision,Recall]
pearsonResult=[PAccuracy1,PPrecision1,PRecall1]
print(AverageResult)
print(pearsonResult)
      
      

In [0]:
       

print("Accuracy",np.average(AverageResult[0]),"Precision",np.average(AverageResult[1]),"Recall",np.average(AverageResult[2]),sep=" ",end="\n")
print("PAccuracy1",np.average(pearsonResult[0]),"PPrecision1",np.average(pearsonResult[1]),"PRecall1",np.average(pearsonResult[2]),sep=" ",end="\n")